In [1]:
import ast
import gensim
import numpy as np
import pandas as pd
import scipy

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [4]:
text = 'определение, содержащее определяемое слово'
tok_text = tokenizer.tokenize(text)
tok_ids = tokenizer.convert_tokens_to_ids(tok_text)

In [5]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias

In [6]:
segments_ids = [0 for x in range(len(tok_text))]

In [7]:
tok_tensor = torch.tensor([tok_ids])
segments_tensor = torch.tensor([segments_ids])

In [8]:
model.to('cuda')
tok_tensor = tok_tensor.to('cuda')
segments_tensor = segments_tensor.to('cuda')

In [9]:
with torch.no_grad():
    encoded_layers, _ = model(tok_tensor)#, segments_tensor)

In [10]:
encoded_layers[-1][0][-2]

tensor([ 3.2099e-01, -2.6179e-01,  1.8232e-01, -3.5652e-02, -2.3675e-01,
         1.1422e-01, -8.9960e-01,  1.2171e+00, -4.9662e-01,  7.4483e-02,
        -3.8856e-02,  2.2836e-01,  1.4959e-01, -7.8604e-01,  9.2562e-02,
         2.6877e-01,  6.4416e-01, -5.4223e-01,  3.1611e-01, -1.0172e+00,
        -1.1422e+00,  8.0431e-01, -5.7586e-01,  5.6152e-01, -6.9567e-01,
        -1.1408e-01, -2.5146e-01,  2.3883e-01, -8.0835e-02, -1.5702e+00,
         6.3383e-01,  2.9404e-01,  1.6067e-01,  2.0980e-01,  8.3308e-02,
         1.1581e-01,  7.8728e-01,  2.4391e-01,  6.7648e-01,  3.0851e-01,
        -1.0920e-01,  2.4244e-01,  1.7484e-01,  4.1751e-02, -4.0461e-01,
        -1.4530e+00, -1.0637e-01, -2.2569e-01, -6.3418e-01, -2.4578e-02,
        -2.9862e-01,  2.8738e-01, -3.9977e-02,  2.4454e-01,  5.8295e-01,
         6.8151e-01,  4.9432e-02, -3.4824e-01,  3.1330e-01, -3.1764e-01,
         1.5635e+00,  1.1231e-01,  1.6233e-02,  5.9727e-02,  1.2818e-01,
         6.5996e-02, -3.9478e-01, -4.6140e-01,  2.5

In [11]:
tok_tensor

tensor([[  555, 71282,   117, 10956, 22605, 17254, 29216,   555, 35415, 87097,
         83455, 19900, 10205, 42638]], device='cuda:0')

In [12]:
class BertToW2v(torch.nn.Module):
    def __init__(self, bert_model_name, lin_shape_in, lin_shape_out): # -, 768, 100
        super(BertToW2v, self).__init__()
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        self.linear_model = torch.nn.Linear(lin_shape_in, lin_shape_out, bias=False) # bias?
        torch.nn.init.uniform_(self.linear_model.weight, -0.1, 0.1)
        
    def forward(self, input_sentence): # ожидаем уже токенизированное предложение
        encoded_layers, _ = self.bert_model(input_sentence) # пока что с конца и для одиночного предложения, переделать для batch'а
        bert_output = encoded_layers[6][0][1]
        #print(bert_output)
        linear_output = self.linear_model(bert_output).unsqueeze(0)
        #print('-----')
        #print(linear_output)
        return linear_output

In [13]:
bw2v = BertToW2v('bert-base-multilingual-cased', 768, 100)
bw2v.to('cuda')

BertToW2v(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInter

In [14]:
train = pd.read_csv('rus_train.csv')[['word', 'def', 'embedding']].rename({'def':'defin'}, axis = 1)

In [15]:
train['embedding'] = train['embedding'].apply(lambda x: ast.literal_eval(x))

In [16]:
loss_function = torch.nn.MSELoss()#CosineEmbeddingLoss()
optimizer = torch.optim.Adam(bw2v.parameters(), lr=0.0001)

In [17]:
%%time

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

n_epochs = 10

for i in range(n_epochs):
    losses = []
    for row in train.itertuples():
        
        defin = row.defin
        defin = '[CLS] [MASK] ' + defin + ' [SEP] [SEP]'
        #print(defin)
        tok_def = tokenizer.tokenize(defin)
        #print(tok_def)
        tok_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tok_def)])
        tok_ids = tok_ids.to('cuda')
        bw2v.zero_grad()
        linear_output = bw2v(tok_ids)
        # linear_output = linear_output#.unsqueeze(0).to('cuda')

        #tensor_ones = torch.ones(1).to('cuda')
        y = torch.tensor(row.embedding).unsqueeze(0).to('cuda')
        loss = loss_function(linear_output, y)#, tensor_ones)
        
        loss.backward()
        optimizer.step()

        losses.append(float(loss.cpu()))
    print(sum(losses) / len(losses))

0.07520924129868492
0.06267246937040037
0.058748655412688304
0.05547126450515635
0.052515053001088136
0.04966808328711496
0.04736516641543288
0.05091659485259444
0.04285484838763255
0.04157638434220638
CPU times: user 52min 26s, sys: 8min 27s, total: 1h 54s
Wall time: 1h 57s


In [18]:
# a = torch.randn(4)
# b = torch.randn(4)
# y = torch.ones(1)

In [19]:
#loss_function(a.unsqueeze_(0), b.unsqueeze_(0), y)

In [20]:
#torch.save(bw2v.state_dict(), 'model_10_epochs.mdl')

In [21]:
test = pd.read_csv('russe-evaluation/russe/evaluation/test.csv')

w2v = gensim.models.KeyedVectors.load_word2vec_format('wiki_raw.norm-sz100-w3-cb0-it1-min5.w2v', binary=True)
lfreq = np.load('rusfreq/lfreq_words.npy')

In [22]:
# word - def - w2v
def_train = pd.read_csv('rus_train.csv')[['word', 'def', 'embedding']]

In [23]:
tv = pd.read_csv('OZHEGOV.TXT', sep = '|', encoding = 'cp1251')

In [24]:
#bw2v = bw2v.eval()
bw2v.eval()

def get_word_embedding(word): # пока что глобальные параметры
    if word not in lfreq:
        try:
            return w2v.get_vector(word)
        except KeyError:
            return np.nan
    else:
        try:
            defin = tv[tv['VOCAB'] == word].reset_index().iloc[0]['DEF']
        except IndexError:
            try:
                return w2v.get_vector(word)
            except KeyError:
                return np.nan
        print('embedding from bert')
        print(word)
        print(defin)
        defin = '[CLS] [MASK] ' + defin + ' [SEP] [SEP]'
        tokens = tokenizer.tokenize(defin)
        tok_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]).to('cuda')
        print(tok_ids)
        #return tok_ids
        with torch.no_grad():
            embedding = bw2v(tok_ids)
        return embedding.to('cpu').numpy()

In [25]:
def get_words_similarity(word1, word2): # также глобальные параметры
    emb1 = get_word_embedding(word1)
    emb2 = get_word_embedding(word2)
    if (np.isnan(np.sum(emb1)) or np.isnan(np.sum(emb2))):
        return np.nan
    return 1 - scipy.spatial.distance.cosine(emb1, emb2)

In [26]:
%%time
test['sim'] = test.apply(lambda x: get_words_similarity(x['word1'], x['word2']), axis = 1)

embedding from bert
абрикос
южное фруктовое дерево семейства розоцветных, дающее сочные плоды с крупной косточкой
tensor([[   101,    103,    571,  58591,  15350,  20216,  57635,  10205, 104925,
          37542,    557,  44666,  10316,  12181,  41102,  10970,    117,  10405,
          54912,  10956,  26176,    556,  75173,  10292,    558,    551,  14360,
          98458,  10384,  59781,  41054,  28530,  10384,    102,    102]],
       device='cuda:0')
embedding from bert
абрикос
южное фруктовое дерево семейства розоцветных, дающее сочные плоды с крупной косточкой
tensor([[   101,    103,    571,  58591,  15350,  20216,  57635,  10205, 104925,
          37542,    557,  44666,  10316,  12181,  41102,  10970,    117,  10405,
          54912,  10956,  26176,    556,  75173,  10292,    558,    551,  14360,
          98458,  10384,  59781,  41054,  28530,  10384,    102,    102]],
       device='cuda:0')
embedding from bert
абрикос
южное фруктовое дерево семейства розоцветных, дающее сочные 

embedding from bert
агитация
активная деятельность по распространению политических идей средствами массовой информации, устными выступлениями с целью оказать воздействие на широкие массы
tensor([[   101,    103,  95301,  72462,  10385,  34112,  10297,    557,  18291,
          99842,  29672, 102423,  10593,  71264,    549,  31710,  34848,  10508,
          97744,  98617,  19033,  29254,    117,    560,  11567,  14350,  88607,
          10508,    558,  35189,  40936,  34402,  11258,  10439,  62594, 104669,
          80605,  10122,  86490,  75442,  10205,  97744,  65198,    102,    102]],
       device='cuda:0')
embedding from bert
агитация
активная деятельность по распространению политических идей средствами массовой информации, устными выступлениями с целью оказать воздействие на широкие массы
tensor([[   101,    103,  95301,  72462,  10385,  34112,  10297,    557,  18291,
          99842,  29672, 102423,  10593,  71264,    549,  31710,  34848,  10508,
          97744,  98617,  19033, 

embedding from bert
баян
разновидность большой гармоники со сложной системой ладов
tensor([[  101,   103, 17257, 18383, 22415, 17100, 31308, 16616, 65412, 25219,
         10648, 10956, 52399, 10316, 37803, 21297, 11292, 26670, 30470,   102,
           102]], device='cuda:0')
embedding from bert
анемия
== малокровие
tensor([[  101,   103,   134,   134, 33489, 10510, 15755, 12686,   102,   102]],
       device='cuda:0')
embedding from bert
малокровие
уменьшение количества красных кровяных клеток и гемоглобина в крови, анемия
tensor([[  101,   103, 39510, 19557, 30578, 71440,   551, 56680, 10970,   551,
         15755, 18971, 12861,   551, 35025, 11899,   549, 65100, 19900, 77162,
         83281,   543, 83191,   117, 69864, 45307, 10385,   102,   102]],
       device='cuda:0')
embedding from bert
анемия
== малокровие
tensor([[  101,   103,   134,   134, 33489, 10510, 15755, 12686,   102,   102]],
       device='cuda:0')
embedding from bert
аномалия
отклонение от нормы, общей закономерност

embedding from bert
акустика
раздел физики изучающий звук
tensor([[  101,   103, 17257, 44030, 88706, 10387, 49299, 60893, 83336,   102,
           102]], device='cuda:0')
embedding from bert
радиоэлектроника
отдельных отраслей науки и техники, развившихся из электроники и радиотехники
tensor([[   101,    103,  52566,  10332,  56680,  20929,  23881,    549,  46806,
            117,  17257,  65682,  64828,  10387, 104121,  19097,    549,  31383,
          10696,  10353,  17785,    102,    102]], device='cuda:0')
embedding from bert
чело
== лоб
tensor([[  101,   103,   134,   134, 30977, 12528,   102,   102]],
       device='cuda:0')
embedding from bert
народность
национальная, народная N2 самобытность; выражение в искусстве наро дных интересов и психического склада
tensor([[  101,   103, 10122, 76292,   117, 41461, 11548,   151, 10729, 14847,
         18766, 29290, 12737,   132, 96195, 44325,   543, 43706, 10205, 21384,
         10316,   545, 10970, 59075, 10433,   549,   556, 13700, 22

tensor([[  101,   103,   134,   134, 50509,   151, 10759,   120,   123,   102,
           102]], device='cuda:0')
embedding from bert
базис
== база N1/2
tensor([[  101,   103,   134,   134, 50509,   151, 10759,   120,   123,   102,
           102]], device='cuda:0')
embedding from bert
балахон
просторный длинный халат
tensor([[  101,   103, 91535, 11092,   545, 20505, 11092,   562, 15522, 10351,
           102,   102]], device='cuda:0')
embedding from bert
балахон
просторный длинный халат
tensor([[  101,   103, 91535, 11092,   545, 20505, 11092,   562, 15522, 10351,
           102,   102]], device='cuda:0')
embedding from bert
народность
национальная, народная N2 самобытность; выражение в искусстве наро дных интересов и психического склада
tensor([[  101,   103, 10122, 76292,   117, 41461, 11548,   151, 10729, 14847,
         18766, 29290, 12737,   132, 96195, 44325,   543, 43706, 10205, 21384,
         10316,   545, 10970, 59075, 10433,   549,   556, 13700, 22710, 19069,
         1868

...

embedding from bert
детище
== ребенок (сын или дочка)
tensor([[   101,    103,    134,    134,    557, 100320,  49062,    113,  22025,
          10880,  10344,  15116,    114,    102,    102]], device='cuda:0')
embedding from bert
жадность
скупость, корыстолюбие
tensor([[   101,    103,    558,  86613,  40780,    117,  80062,  36257,  10752,
          21190, 108306,    102,    102]], device='cuda:0')
embedding from bert
шествие
торжественное прохождение, процессия, манифестация
tensor([[  101,   103, 11663, 60872, 28547, 25196, 12621, 12709, 42940, 32450,
           117, 45234, 11502,   117, 97744, 10656, 42695, 15535, 12942,   102,
           102]], device='cuda:0')
embedding from bert
кортеж
торжественное шествие, процессия, выезд
tensor([[  101,   103, 11663, 60872, 28547, 25196, 12621, 54675, 55763,   117,
         45234, 11502,   117, 96195, 58468,   102,   102]], device='cuda:0')
embedding from bert
шествие
торжественное прохождение, процессия, манифестация
tensor([[  101,   103,

embedding from bert
флора
растительный мир
tensor([[  101,   103,   557, 31016, 95144, 29345,   102,   102]],
       device='cuda:0')
embedding from bert
белогвардеец
контрреволюционер, сражавшийся против советской власти в рядах белой гвардии
tensor([[   101,    103,  33023,  46672, 110442,  77482,  86228,  28588,    117,
            558,  11079,  65718,  74478,  13488,  55086,  21059,    543,  50114,
          10353,  32609,  11292,  50848,    102,    102]], device='cuda:0')
embedding from bert
диктатура
ничем не ограниченная власть, опирающаяся на прямое насилие
tensor([[  101,   103, 19544, 52086, 10375,   555, 69330, 10656, 16580, 11548,
         48594,   117,   555, 89534, 33645, 10625, 10122, 75552, 10205, 32001,
         16878, 10205,   102,   102]], device='cuda:0')
embedding from bert
щелчок
отрывистый удар чем-нибудь твердым (обычно об ударе разгибаемым пальцем); негромкий звук короткого и резкого удара
tensor([[   101,    103,  10332,  77851, 105728,  10384,  57514,  18658,

embedding from bert
окунь
рыба с красноватыми нижними плавниками
tensor([[   101,    103,    557,  10292,  13390,    558,    551,  56680,  36481,
         106594,  19544,  85307,  10191,    556,  29952,  37483,    102,    102]],
       device='cuda:0')
embedding from bert
щука
хищная пресноводная рыба отряда лососеобразных
tensor([[  101,   103,   562, 10191, 18565, 11548, 38494, 24905, 45597, 11548,
           557, 10292, 13390, 66414, 30977, 30604, 13110, 33276, 34556, 10970,
           102,   102]], device='cuda:0')
embedding from bert
щука
хищная пресноводная рыба отряда лососеобразных
tensor([[  101,   103,   562, 10191, 18565, 11548, 38494, 24905, 45597, 11548,
           557, 10292, 13390, 66414, 30977, 30604, 13110, 33276, 34556, 10970,
           102,   102]], device='cuda:0')
embedding from bert
щука
хищная пресноводная рыба отряда лососеобразных
tensor([[  101,   103,   562, 10191, 18565, 11548, 38494, 24905, 45597, 11548,
           557, 10292, 13390, 66414, 30977, 30604, 1

embedding from bert
сланец
слоистые горные породы, обладающие способностью раскалываться на тонкие пластины, слои
tensor([[   101,    103,  52399,  26891,  34460,  10205, 104812,  11194,  41436,
          50703,    117,  58742,  36069,  23249,    558,  94131,  10593,    557,
          18291,  48225,  72060,  10625,  10122,  11663,  25751,  10205,    556,
          38445,  93526,    117,  52399,  26891,    102,    102]],
       device='cuda:0')
embedding from bert
надзиратель
должностное лицо, которое занимается надзором за кем-чем-нибудь
tensor([[   101,    103,  80323,  11567,  12621, 107643,    117,  26379, 104052,
          12614,  48645,  10364,  10234,    551,  12579,    118,  18658,    118,
          19544,  91369,    102,    102]], device='cuda:0')
embedding from bert
экспромт
речь, стихотворение, музыкальное произведение, создаваемые без подготовки, в момент произнесения, исполнения, импровизация
tensor([[   101,    103,    557,  75865,    117,  15888,  13008,  11016,  29973,
 

tensor([[  101,   103,   134,   134,   542, 54869, 58056, 68473, 59205,   102,
           102]], device='cuda:0')
embedding from bert
волейбол
спортивная командная игра в мяч, который участники перебрасывают через высокую сетку, стараясь, чтобы он коснулся земли на площадке соперников
tensor([[   101,    103,  44505,  72462,  10385,  33162,  11548,  28940,    543,
            553,  65891,    117,  12968, 110800,  61381,  59373,  56680,  69300,
          12798,  96195,  45135,  14611,  10277,  48038,    117,  15888,  16682,
          39269,    117,  15692,  11060,  59781,  10513, 109075,  10625,  32369,
          10122,  72214,  10746,  11557,  10956,  29633,  17167,    102,    102]],
       device='cuda:0')
embedding from bert
ром
крепкий алкогольный напиток, получаемый из сока сахарного тростника
tensor([[   101,    103,    551,  14348,  46953,  10384,  21720,  18632,  23323,
          10122,  80465,  11899,    117,  91258,  64916,  50911,  10387,  10956,
          10521,  10868, 1048

In [27]:
get_word_embedding('хомут')

embedding from bert
хомут
приспособление кольцевой формы для скрепления, соединения чего-нибудь, скоба Spec
tensor([[   101,    103,  10913,  10513,  97205,  20539,  90363,  10851,  16748,
          11292,  38366,  10520,    558,  10510,  14348,  70675,    117, 110222,
          22121,    118,  19544,  91369,    117,    558,  11623,  13390,  46361,
          15953,    102,    102]], device='cuda:0')


array([[ 0.10772858, -0.12910084, -0.10418984, -0.27090928,  0.1397202 ,
        -0.04462191, -0.3270972 , -0.05341005,  0.4920879 , -0.15564972,
        -0.29493728,  0.04002178, -0.5093604 ,  0.6002498 , -0.24913925,
         0.23754524,  0.17779845, -0.07484733, -0.09382335,  0.19110395,
         0.15283763,  0.1425031 , -0.10458218,  0.43501893, -0.23809889,
        -0.12148573,  0.297172  ,  0.08231001, -0.34975505,  0.30733693,
        -0.11360589,  0.42485255,  0.11589998,  0.02986502,  0.41398227,
        -0.22825266,  0.06865311, -0.16299573,  0.2679476 ,  0.1500406 ,
        -0.1333563 , -0.16489697, -0.17703989,  0.06727467, -0.52598315,
         0.09838584, -0.09960897, -0.08066987, -0.3483227 ,  0.0849028 ,
         0.25566724, -0.24871974,  0.4539797 ,  0.45186582, -0.05302969,
         0.10554338, -0.10896063, -0.02429081,  0.13353933,  0.49778175,
         0.35880762,  0.538044  ,  0.17416802,  0.21192802,  0.43109706,
         0.3191654 ,  0.70212424,  0.7735244 ,  0.1

In [28]:
test = test[test['sim'].notnull()]

In [31]:
test.to_csv('russe-evaluation/russe/evaluation/bert_test_10_mse.csv', index=False)

In [30]:
torch.save(bw2v.state_dict(), 'model_10_epochs_mse.mdl')